In [75]:
import random
import os
import json
from functools import wraps

## Задание №1
```
📌Создайте функцию-замыкание, которая запрашивает два целых числа:
    ○ от 1 до 100 для загадывания,
    ○ от 1 до 10 для количества попыток
📌Функция возвращает функцию, которая через консоль просит угадать загаданное число за указанное число попыток.  
```

In [53]:
import sys
sys.path.insert(0, r"..\prac_6")
import task2

In [54]:
def initGuess(upperLimit, attLimit):
    def inner():
        return task2.guessNum(1, upperLimit, attLimit)

    return inner

In [55]:
guessFunc = initGuess(100, 10)
guessFunc()

Угадайте число от 1 до 100
Больше, чем 50
Больше, чем 30
Меньше, чем 75
Меньше, чем 63
Меньше, чем 58
Больше, чем 54
Больше, чем 56
<< 57 >>  Ура, Вы угадали!


True

## Задание №2
```
📌Дорабатываем задачу 1.
📌Превратите внешнюю функцию в декоратор.
📌Он должен проверять входят ли переданные в функцию-угадайку числа в диапазоны [1, 100] и [1, 10].
📌Если не входят, вызывать функцию со случайными числами из диапазонов.
```

In [86]:
def checkParams(func):
    @wraps(func)
    def wrapper(*args):
        if args[0] < 1 or args[0] > 100 or args[1] < 0 or args[1] > 10:
            return func(random.randint(1, 100), random.randint(1, 10))
        return func(*args)

    return wrapper

In [61]:
@checkParams
def guessFunc(upperLimit, attLimit):
    return task2.guessNum(1, upperLimit, attLimit)

guessFunc(10, 3)

Угадайте число от 1 до 10


<< 5 >>  Ура, Вы угадали!


True

## Задание №3
```
📌Напишите декоратор, который сохраняет в json файл параметры декорируемой функции и результат, который она возвращает. При повторном вызове файл должен расширяться, а не перезаписываться.
📌Каждый ключевой параметр сохраните как отдельный ключ json словаря.
📌Для декорирования напишите функцию, которая может принимать как позиционные, так и ключевые аргументы.
📌Имя файла должно совпадать с именем декорируемой функции.
```

In [65]:
import inspect

def getArgsStr(func, *args, **kwargs) -> str:
    """
    Возвращает строку с аргументами функции, с учётом значений по умолчанию.
    Значения именованных аргументов идут в порядке, заданном в определении функции.
    Значения не заданных в определении именованных аргументов отсортированы по именам.

    Args:
        func: Функция, в которую передаются параметры

    Returns:
        Строка значений аргументов, разделённых запятыми
    """
    argsData = inspect.getfullargspec(func)
    res = []
    
    res.append(", ".join(map(str, args)))
    
    res.append(", ".join((
        str(kwargs.setdefault(
            argsData.args[len(args) + i],
            argsData.defaults[len(argsData.defaults) - len(argsData.args) + len(args):][i]
        ))
        for i in range(len(argsData.args) - len(args))
    )))
    
    res.append(", ".join((
        str(kwargs.setdefault(k, argsData.kwonlydefaults[k]))
        for k in argsData.kwonlyargs
    )))
    
    res.append(", ".join((
        str(kwargs[k])
        for k in sorted(kwargs.keys())
        if k not in argsData.args + argsData.kwonlyargs
    )))
    
    return ", ".join(filter(bool, res))

In [83]:
def cacheToJson(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        fName = func.__name__ + ".json"
        adding = True if os.path.exists(fName) else False
        
        with open(fName, "r+" if adding else "w", encoding="utf-8") as jsonFile:
            if adding:
                hashDict = json.load(jsonFile)
            else:
                hashDict = {}

            argsStr = getArgsStr(func, *args, **kwargs)
            if argsStr in hashDict:
                return hashDict[argsStr]
            else:
                res = func(*args, **kwargs)

                if adding:
                    jsonFile.seek(jsonFile.tell() - 3)
                
                jsonFile.write(
                    ("," if adding else "{") \
                    + json.dumps({argsStr: res}, ensure_ascii=False, indent=2)[1:]
                )

        return res
    
    return wrapper

In [18]:
@cacheToJson
def task3_func(a, b, c=1, f=1, p=1):
    print("Выполнение")
    return ((a + b + c) * f) ** p

In [44]:
task3_func(5, 3, p=3)

729

In [63]:
@cacheToJson
def task3_func1(a, b, f=2, *args, **kwargs):
    return (a + b) * f + sum(args + tuple(kwargs.values()))

In [67]:
task3_func1(1, 2, 3, y=7, x=2)

18

In [46]:
import functools

@functools.cache
def task3_func2(a, b, f=2, *args, **kwargs):
    print("Выполнение")
    return (a + b) * f + sum(args + tuple(kwargs.values()))

In [49]:
task3_func2(1, 2, f=7, q=3)

24

In [31]:
a = {'y': 1, 'x': 2}
sorted(a.keys())

['x', 'y']

## Задание №4
```
📌 Создайте декоратор с параметром.
📌 Параметр - целое число, количество запусков декорируемой функции.
```

In [92]:
def repeat(num):
    def inner(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            results = []
            for _ in range(num):
                results.append(func(*args, **kwargs))
            return results
        return wrapper
    return inner

In [14]:
@repeat(5)
def task4_func():
    print("Hello world!")

task4_func()

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


[None, None, None, None, None]

## Задание №5
```
📌 Объедините функции из прошлых задач.
📌 Функцию угадайку задекорируйте:
    ○ декораторами для сохранения параметров,
    ○ декоратором контроля значений и
    ○ декоратором для многократного запуска.
📌 Выберите верный порядок декораторов.
```

In [90]:
@checkParams
@cacheToJson
@repeat(5)
def guessFunc(upperLimit, attLimit):
    return task2.guessNum(1, upperLimit, attLimit)

In [95]:
guessFunc(90, 10)

[True, False, True, True, False]

## Задание №6
```
📌 Доработайте прошлую задачу добавив декоратор wraps в каждый из декораторов.
```